::::
:::{thebe-button}
:::
::::

# Find tracks

In [ ]:
from __future__ import annotations

from collections.abc import Callable, Iterable
from datetime import datetime
from inspect import Signature

from boilercore.paths import ISOLIKE, dt_fromisolike
from boilercv_pipeline.dfs import limit_group_size
from boilercv_pipeline.models.column import Col, LinkedCol, convert, rename
from boilercv_pipeline.models.deps import get_slices
from boilercv_pipeline.models.df import GBC
from boilercv_pipeline.models.params.types import DfOrS_T
from boilercv_pipeline.models.subcool import const
from boilercv_pipeline.palettes import cat10, cool
from boilercv_pipeline.plotting import get_cat_colorbar
from boilercv_pipeline.sets import inspect_video
from boilercv_pipeline.stages import find_objects, get_thermal_data
from boilercv_pipeline.stages.find_tracks import FindTracks as Params
from boilercv_pipeline.units import U
from dev.docs.nbs import init
from devtools import pprint
from matplotlib.figure import Figure
from matplotlib.pyplot import subplot_mosaic, subplots
from more_itertools import one, only
from numpy import diff, gradient, linalg, pi, vectorize
from pandas import DataFrame, Series, melt, merge_ordered, read_hdf
from seaborn import scatterplot
from trackpy import link, quiet

from boilercv.correlations import GROUPS
from boilercv.correlations import beta as correlations_beta
from boilercv.correlations import nusselt as correlations_nusselt
from boilercv.correlations.types import Corr
from boilercv.data import FRAME, TIME, VIDEO
from boilercv.dimensionless_params import (
    fourier,
    jakob,
    kinematic_viscosity,
    nusselt,
    prandtl,
    reynolds,
    thermal_diffusivity,
)

quiet()

PARAMS = None

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(
        context=init(),
        include_patterns=const.nb_include_patterns,
        slicer_patterns=const.nb_slicer_patterns,
    )
context = params.context
params.set_display_options()
data = params.data
C = params.cols

slices = get_slices(one(params.filled_slicers))
objects_path = one(params.objects)
objects = (
    read_hdf(objects_path)
    .set_index(C.frame())
    .loc[slices.get(FRAME, slice(None)), :]
    .reset_index()
)
frames = slices.get(FRAME, slice(None))
OC = find_objects.Cols()

with inspect_video(one(params.filled)) as src:
    filled = src[VIDEO]
    slices = get_slices(one(params.filled_slicers))
    objects = (
        read_hdf(objects_path)
        .set_index(C.frame())
        .loc[frames, :]
        .reset_index()
        .assign(**{
            C.time_elapsed(): lambda df: src.coords[TIME].sel({
                FRAME: df[C.frame()].values
            })
        })
    )
    step_time = diff(src[FRAME].sel({FRAME: objects[C.frame()].unique()})[:2])[0]

dfs = only(params.dfs)

thermal = read_hdf(params.deps.thermal)
TC = get_thermal_data.Cols()


def get_time() -> datetime:
    """Get time from path."""
    if match := ISOLIKE.search(objects_path.stem):
        return dt_fromisolike(match)
    else:
        raise ValueError("No time found in path.")


time = get_time()
subcooling = thermal.set_index(TC.time())[TC.subcool()][time]

frames_per_step = frames.step
time_per_frame = step_time / frames_per_step

cols_to_link = [C.frame, OC.x_tp, OC.y_tp]

# ! To find (202 - 96)
# %matplotlib widget
# from boilercv_pipeline.sets import get_dataset
# _, ax = subplots()
# ax.imshow(get_dataset("2024-07-18T17-44-35", stage="large_sources")[VIDEO].sel(frame=0))

M_PER_PX = U.convert(3 / 8, "in", "m") / (202 - 96)
# PX_PER_M = 20997.3753
U.define(f"px = {M_PER_PX} m")
U.define(f"frames = {time_per_frame} s")

# Linking
SEARCH_RANGE = 5 * frames_per_step
"""Pixel range to search for the next bubble."""
MEMORY = 5 * frames_per_step
"""Frames to remember a bubble."""

# Track tuning
Y_SURFACE_THRESHOLD = U.convert(280, "px", "m")
"""Vertical position of bubble centroids considered attached to the surface."""
Y_DEPARTURE_THRESHOLD = U.convert(300, "px", "m")
"""Vertical position of bubble centroids considered to have departed the surface."""
MINIMUM_LIFETIME = 0.005  # s
"""Minimum bubble lifetime to consider."""

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3


# Plotting
groups = {C.corr[k](): v for k, v in GROUPS.items() if k in C.corr}
"""Groups for mapping to correlations in data."""
GROUP_DRAW_ORDER = ["Group 2", "Group 4", "Group 3", "Group 1"]
"""Order to draw groups."""
GROUP_ORDER = sorted(GROUP_DRAW_ORDER)
"""Order to show groups in legend."""
GROUP_SORTER = vectorize(GROUP_DRAW_ORDER.index)
"""Sorter for groups."""
CORRELATIONS_PALETTE = cat10
"""For plotting one approach."""
TRACKS_PALETTE = cool
"""For plotting the other approach."""
MAX_FOURIER = 0.005
"""Maximum Fourier number to plot."""
MAX_BETA = 1.05
"""Maximum dimensionless bubble diameter to plot."""
MAX_NUSSELT = 1000
"""Maximum Nusselt number to plot."""
TRACKS_ALPHA = 0.4
"""Transparency of the tracks."""
TRACKS_SIZE = 10
"""Size of the tracks."""
MAX_BETA_MAE = 0.3
"""Maximum mean absolute error of beta to plot."""
MAX_NUSSELT_ERR = 12000
"""Maximum mean absolute error of nusselt to plot."""
WIDTH_SCALE = 1.89
"""Width to scale plots by."""
HEIGHT_SCALE = 1.30
"""Width to scale plots by."""


def scale_figure(fig: Figure, width: float = WIDTH_SCALE, height: float = HEIGHT_SCALE):
    """Scale up figure size."""
    fig.set_figwidth(width * fig.get_figwidth())
    fig.set_figheight(height * fig.get_figheight())


def get_init(ser: Series[float]) -> float:
    """Get initial value of a series."""
    return ser.head(int(MINIMUM_LIFETIME // step_time) or 1).median()


def get_delta(df: DataFrame, c: LinkedCol) -> Series[float]:
    """Get position time delta across frames."""
    return df.groupby(C.bub(), **GBC)[[c.source()]].diff().fillna(0) / step_time


def query_lifetime(df: DataFrame) -> DataFrame:
    """Filter bubbles by lifetime."""
    return (
        df.rename(columns={C.bub_visible(): (temp_name := C.bub_visible.no_unit.name)})
        .query(f"`{temp_name}` > {MINIMUM_LIFETIME}")
        .rename(columns={temp_name: C.bub_visible()})
    )


beta_correlations = correlations_beta.get_correlations()
nusselt_correlations = correlations_nusselt.get_correlations()
constants = {
    "Ja": jakob(
        liquid_density=LIQUID_DENSITY,
        vapor_density=VAPOR_DENSITY,
        liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        subcooling=subcooling,
        latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
    ),
    "Pr": prandtl(
        dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "alpha": thermal_diffusivity(
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        density=LIQUID_DENSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    ),
    "pi": pi,
    "liquid_kinematic_viscosity": kinematic_viscosity(
        density=LIQUID_DENSITY, dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY
    ),
    "liquid_thermal_diffusivity": thermal_diffusivity(
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        density=LIQUID_DENSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    ),
    "liquid_prandtl": prandtl(
        dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "bubble_jakob": jakob(
        liquid_density=LIQUID_DENSITY,
        vapor_density=VAPOR_DENSITY,
        liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        subcooling=subcooling,
        latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
    ),
}


def get_corrs(df: DataFrame, kind: Corr) -> DataFrame:
    """Get correlations."""
    return df.assign(**{
        C.corr[label](): (
            corr.expr(**{
                kwd: value
                for kwd, value in {
                    **constants,
                    "Re_b": df[C.bub_reynolds()],
                    "Re_b0": df[C.bub_reynolds0()],
                    "Fo_0": df[C.bub_fourier()],
                    **({} if kind == "beta" else {"beta": df[C.bub_beta()]}),
                }.items()
                if kwd in Signature.from_callable(corr.expr).parameters
            })
        )
        for label, corr in (
            beta_correlations if kind == "beta" else nusselt_correlations
        ).items()
    })


# def get_error(df: DataFrame, kind: Corr) -> DataFrame:
#     """Get absolute error."""
#     corrs = [c() for c in C.corr.values()]
#     exp = C.bub_beta() if kind == "beta" else C.bub_nusselt()
#     return (
#         df.set_index([C.bub_fourier(), C.bub()])[[exp, *corrs]]
#         .pipe(lambda df: df.assign(**{c: abs(df[exp] - df[c]) for c in corrs}))
#         .reset_index()
#     )


def get_error(df: DataFrame, kind: Corr) -> DataFrame:
    """Get relative error."""
    corrs = [c() for c in C.corr.values()]
    exp = C.bub_beta() if kind == "beta" else C.bub_nusselt()
    return (
        df.set_index([C.bub_fourier(), C.bub()])[[exp, *corrs]]
        .pipe(lambda df: df.assign(**{c: abs(df[exp] - df[c]) / df[c] for c in corrs}))
        .reset_index()
    )


def set_group_legend(ax):
    """Set legend for correlation groups."""
    legend = ax.legend(
        [
            {lab: h for h, lab in zip(*ax.get_legend_handles_labels(), strict=False)}[
                lab
            ]
            for lab in GROUP_ORDER
        ],
        GROUP_ORDER,
    )
    for handle in legend.legend_handles:  # pyright: ignore[reportAttributeAccessIssue]
        handle.set_alpha(1.0)


def preview(
    df: DfOrS_T,
    cols: Iterable[Col] | None = None,
    index: Col | None = None,
    f: Callable[[DfOrS_T], DfOrS_T] | None = None,
    ncol: int = 0,
) -> DfOrS_T:
    """Preview a dataframe in the notebook."""
    # fmt: off
    if df.empty:
        display(df)
        return df
    if isinstance(df, Series):
        def _f(df): return (f(df) if f else df).head(16)
    elif C.bub() in df.columns:
        def _f(df): return (_df := f(df) if f else df).groupby(C.bub(), **GBC)[_df.columns].head(4).head(16)  # pyright: ignore[reportRedeclaration]
    else:
        def _f(df): return (f(df) if f else df).head(4).head(16)
    # fmt: on
    df = params.preview(cols=cols, df=df, index=index, f=_f, ncol=ncol)  # pyright: ignore[reportArgumentType]
    return df


pprint(params)

In [ ]:
data.dfs.tracks = preview(
    ncol=12,
    cols=C.tracks,
    df=link(
        # ? TrackPy expects certain column names
        f=objects.rename(columns={c(): c.source.raw for c in cols_to_link}),  # pyright: ignore[reportCallIssue]
        search_range=SEARCH_RANGE,
        memory=MEMORY,
    )
    .pipe(rename, cols_to_link)  # ? Back to our names
    .pipe(C.bub.rename)
    .assign(**{
        C.bub_visible_frames(): lambda df: (
            df.groupby(C.bub(), **GBC)[C.bub()].transform("count") * frames_per_step
        )
    })
    .pipe(convert, [C.x, C.y, C.diameter, C.radius_of_gyration], U)
    .sort_values(
        [C.bub_visible_frames(), C.bub(), C.frame()], ascending=[False, True, True]
    )
    .assign(**{
        C.bub(): (lambda df: df.groupby(C.bub(), **GBC).ngroup()),
        C.u(): lambda df: get_delta(df, C.u),
        C.v(): lambda df: get_delta(df, C.v),
        C.distance(): lambda df: linalg.norm(df[[C.u(), C.v()]].abs(), axis=1),
    })
    .pipe(convert, [C.bub_visible], U),
)
print(f"Found {data.dfs.tracks[C.bub()].nunique()} bubbles")

In [ ]:
data.dfs.bubbles = preview(
    ncol=12,
    cols=C.bubbles,
    # ? Find rows corresponding to stagnant or invalid bubbles
    df=data.dfs.tracks.pipe(lambda df: df[df[C.bub_visible()] > MINIMUM_LIFETIME])
    .groupby(C.bub(), **GBC)[data.dfs.tracks.columns]
    .apply(
        # ? Don't assign any other columns until invalid rows have been filtered out
        lambda df: df.assign(**{
            "bubble_visible_y": lambda df: df[C.y()].pipe(get_init),
            # ? Initial y position is close to the surface
            "began": lambda df: df["bubble_visible_y"] > Y_SURFACE_THRESHOLD,
            # ? When the bubble gets far enough away from the surface
            "departed": lambda df: df[C.y()] < Y_DEPARTURE_THRESHOLD,
        })
    )
    # ? Filter out invalid rows
    .pipe(lambda df: df[df["began"] & df["departed"]])
    .pipe(limit_group_size, C.bub(), 1)
    # ? Groupby again after filtering out invalid rows
    .groupby(C.bub(), **GBC)[data.dfs.tracks.columns]
    # ? Now columns that depend on the initial row (*.iat[0]) can be assigned
    .apply(
        lambda df: df.assign(**{
            C.bub_time(): (
                lambda df: df[C.time_elapsed()] - df[C.time_elapsed()].iat[0]
            ),
            C.bub_lifetime(): lambda df: (
                df[C.bub_time()].iat[-1] - df[C.bub_time()].pipe(get_init)
            ),
            C.bub_t0(): lambda df: df[C.bub_time()].pipe(get_init),
            C.bub_x0(): lambda df: df[C.x()].pipe(get_init),
            C.bub_y0(): lambda df: df[C.y()].pipe(get_init),
            C.bub_d0(): lambda df: df[C.diameter()].pipe(get_init),
            C.bub_u0(): lambda df: df[C.u()].pipe(get_init),
            C.bub_v0(): lambda df: df[C.v()].pipe(get_init),
            C.max_diam(): lambda df: df[C.diameter()].max(),
            C.diam_rate_of_change(): lambda df: gradient(
                df[C.diameter()], df[C.bub_time()]
            ),
        })
    ),
)
print(f"{data.dfs.bubbles[C.bub()].nunique()} bubbles remain")

In [ ]:
data.plots.multi, axs = subplot_mosaic([
    [C.y(), C.v()],
    [C.diameter(), C.diam_rate_of_change()],
])
scale_figure(data.plots.multi)
for plot, ax in axs.items():
    palette, tracks_data = get_cat_colorbar(
        ax, C.bub(), TRACKS_PALETTE, data.dfs.bubbles
    )
    scatterplot(
        ax=ax,
        edgecolor="none",
        s=TRACKS_SIZE,
        alpha=TRACKS_ALPHA,
        x=C.bub_time(),
        y=plot,  # pyright: ignore[reportArgumentType] 1.1.356
        hue=C.bub(),
        legend=False,
        palette=palette,
        data=tracks_data,
    )

In [ ]:
data.dfs.dst = preview(
    cols=C.dests,
    df=data.dfs.bubbles.assign(**{
        C.bub_reynolds(): lambda df: reynolds(
            velocity=abs(df[C.v()]),
            characteristic_length=df[C.diameter()],
            kinematic_viscosity=constants["liquid_kinematic_viscosity"],
        ),
        C.bub_reynolds0(): lambda df: df.groupby(C.bub(), **GBC)[
            C.bub_reynolds()
        ].transform(get_init),
        C.bub_fourier(): fourier(
            initial_bubble_diameter=data.dfs.bubbles[C.bub_d0()],
            liquid_thermal_diffusivity=constants["liquid_thermal_diffusivity"],
            time=data.dfs.bubbles[C.bub_time()],
        ),
        C.bub_nusselt(): nusselt(  # Nu_c
            heat_transfer_coefficient=-(
                2
                * VAPOR_DENSITY
                * LATENT_HEAT_OF_VAPORIZATION
                / subcooling
                * data.dfs.bubbles[C.diam_rate_of_change()]
            ),
            characteristic_length=data.dfs.bubbles[C.diameter()],
            thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        ),
        C.bub_beta(): (lambda df: df[C.diameter()] / df[C.bub_d0()]),
    }),
)

In [ ]:
data.dfs.beta = preview(
    cols=C.corr_beta, df=data.dfs.dst.pipe(get_corrs, kind="beta"), ncol=6
)
data.dfs.nusselt = preview(
    cols=C.corr_nusselt, df=data.dfs.dst.pipe(get_corrs, kind="nusselt"), ncol=6
)

In [ ]:
data.dfs.beta_err = preview(
    cols=C.corr_beta, df=data.dfs.beta.pipe(get_error, kind="beta"), ncol=6
)

data.plots.beta_err, ax = subplots()
ax.set_ylabel(C.bub_beta())
ax.set_xscale("log")
ax.set_yscale("log")
scatterplot(
    ax=ax,
    edgecolor="none",
    hue_order=GROUP_DRAW_ORDER,
    s=TRACKS_SIZE,
    alpha=TRACKS_ALPHA,
    palette="tab10",
    x=C.bub_fourier(),
    y=C.bub_beta(),
    hue="Group",
    data=(
        melt(
            data.dfs.beta_err.set_index(C.bub_fourier())[
                [c() for c in C.corr.values()]
            ],
            var_name="Correlation",
            value_name=C.bub_beta(),
            value_vars=[c() for c in C.corr.values()],
            ignore_index=False,
        )
        .assign(**{
            C.bub_fourier(): lambda df: df.index,
            "Group": lambda df: df["Correlation"].map(groups),
        })
        .reset_index(drop=True)
        .sort_values("Group", key=GROUP_SORTER)
    ),
)
set_group_legend(ax)

In [ ]:
data.dfs.nusselt_err = preview(
    cols=C.corr_nusselt, df=data.dfs.nusselt.pipe(get_error, kind="nusselt"), ncol=6
)


data.plots.nusselt_err, ax = subplots()
ax.set_ylabel(C.bub_nusselt())
ax.set_ylim(1e-5, 1e5)
ax.set_xscale("log")
ax.set_yscale("log")
scatterplot(
    ax=ax,
    edgecolor="none",
    hue_order=GROUP_DRAW_ORDER,
    s=TRACKS_SIZE,
    alpha=TRACKS_ALPHA,
    palette="tab10",
    x=C.bub_fourier(),
    y=C.bub_nusselt(),
    hue="Group",
    data=(
        melt(
            data.dfs.nusselt_err.set_index(C.bub_fourier())[
                [c() for c in C.corr.values()]
            ],
            var_name="Correlation",
            value_name=C.bub_nusselt(),
            value_vars=[c() for c in C.corr.values()],
            ignore_index=False,
        )
        .assign(**{
            C.bub_fourier(): lambda df: df.index,
            "Group": lambda df: df["Correlation"].map(groups),
        })
        .reset_index(drop=True)
        .sort_values("Group", key=GROUP_SORTER)
    ),
)
set_group_legend(ax)

In [ ]:
beta_mae = data.dfs.beta.groupby(C.bub(), **GBC)[[c() for c in C.corr.values()]].apply(
    lambda df: df.sum() / len(df)
)
beta_mae_pivoted = (
    melt(
        beta_mae.set_index(C.bub()),
        var_name="Correlation",
        value_name=C.bub_beta(),
        value_vars=[c() for c in C.corr.values()],
        ignore_index=False,
    )
    .assign(**{
        C.bub(): lambda df: df.index,
        "Group": lambda df: df["Correlation"].map(groups),
    })
    .reset_index(drop=True)
    .sort_values("Group")
)
nusselt_mae = data.dfs.nusselt.groupby(C.bub(), **GBC)[
    [c() for c in C.corr.values()]
].apply(lambda df: df.sum() / len(df))
nusselt_mae_pivoted = (
    melt(
        nusselt_mae.set_index(C.bub()),
        var_name="Correlation",
        value_name=C.bub_nusselt(),
        value_vars=[c() for c in C.corr.values()],
        ignore_index=False,
    )
    .assign(**{
        C.bub(): lambda df: df.index,
        "Group": lambda df: df["Correlation"].map(groups),
    })
    .reset_index(drop=True)
    .sort_values("Group")
)

data.plots.mae, ax = subplots()
ax.set_xscale("log")
ax.set_yscale("log")
scatterplot(
    ax=ax,
    edgecolor="none",
    hue_order=GROUP_DRAW_ORDER,
    s=TRACKS_SIZE,
    alpha=TRACKS_ALPHA,
    palette="tab10",
    x=C.bub_beta(),
    y=C.bub_nusselt(),
    hue="Group",
    data=(
        merge_ordered(
            beta_mae_pivoted.drop(columns="Group").set_index(C.bub()),
            nusselt_mae_pivoted.drop(columns="Group").set_index(C.bub()),
            on=[C.bub(), "Correlation"],
        )
        .assign(**{"Group": lambda df: df["Correlation"].map(groups)})
        .sort_values("Group", key=GROUP_SORTER)
    ),
)
set_group_legend(ax)